# Pennsylvania Select LiDAR tiles

The project examines slope rasters derived from LiDAR to find RCH. Evidence of RCH is more often found in forested areas where ground has not been plowed or built on and so it may be useful to examine only liDAR tiles that contain forested areas.

## Inputs

### LiDAR tiles shapefile

This is from FEMA, projected to crs: 32129
Get LiDAR las files from https://coast.noaa.gov/dataviewer/ Data Access Viewer

Bulk download and information
https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/2574/index.html

tile index:
https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/2574/tileindex.zip


lidar_tiles_path = "/content/drive/MyDrive/crane_pennsylvania/pa2006_2008_pamap_south_index/pa2006_2008_pamap_south_index.shp"

### Study Area

This is a shape encompassing Pennsylvania's Michaux State Forest and nearby forested areas:

study_area_path = "/content/drive/MyDrive/crane_pennsylvania/pa_michaux_forest_area_rch/study_area_shapefile/michaux_state_forest_study_area_32129.shp"

## Outputs

A shapefile to contain LiDAR tiles that are in forested areas:

lidar_tiles_forested_polys_fp = "/content/drive/MyDrive/crane_maryland/polys/2012_fema_md_forested.shp"

## Method

Examine each LiDAR tile polygon and see if it overlaps the Geopandas Geoseries of forested polygons.

```
if (land_use_geoseries.overlaps(pred_poly).any()==True) or (land_use_geoseries.contains(pred_poly).any()==True) or (land_use_geoseries.within(pred_poly).any()==True):
```

## Notes

Using geoseries.overlap results in not including some LiDAR tile polygons that are entirely inside of a forested area. The overlap method must not work for that. https://geopandas.readthedocs.io/en/latest/docs/reference/api/geopandas.GeoSeries.overlaps.html "Geometries overlaps if they have more than one but not all points in common." Adding .contains and .within as well.


In [3]:
!pip install geopandas

  Using cached geopandas-0.10.2-py2.py3-none-any.whl (1.0 MB)
     |████████████████████████████████| 16.7 MB 26.8 MB/s 
     |████████████████████████████████| 6.3 MB 41.2 MB/s 


In [ ]:
#QGIS
#Filter DistrictNu = 1 'Michaux
#Reproject 32129

"""

import os
import geopandas as gpd

lidar_tiles_path = "/content/drive/MyDrive/crane_pennsylvania/pa2006_2008_pamap_south_index/pa2006_2008_pamap_south_index.shp"
study_area_path = "/content/drive/MyDrive/crane_pennsylvania/pa_michaux_forest_area_rch/study_area_shapefile/michaux_state_forest_study_area_32129.shp"
lidar_tiles_forested_polys_fp = "/content/drive/MyDrive/crane_pennsylvania/pa_michaux_forest_area_rch/polys/pamap_2006-2008_lidar_michaux_area.shp"

# Create an empty geopandas GeoDataFrame for duplicates
lidar_tiles_forested_polys_df = gpd.GeoDataFrame()
lidar_tiles_forested_polys_df['Index'] = None
lidar_tiles_forested_polys_df['Name'] = None
lidar_tiles_forested_polys_df['URL'] = None
lidar_tiles_forested_polys_df['geometry'] = None

lidar_tiles_forested_polys_df.crs = ('EPSG:32129')

if os.path.exists(lidar_tiles_path) and os.path.exists(study_area_path):
    lidar_tiles_polys = gpd.read_file(lidar_tiles_path)
    land_use_polys = gpd.read_file(study_area_path)
    print("crs", lidar_tiles_polys.crs,land_use_polys.crs )
    print(type(land_use_polys))
    print(list(land_use_polys))
    print("lidar_tiles_polys",lidar_tiles_polys.shape, "land_use_polys",land_use_polys.shape)
    land_use_geoseries = land_use_polys['geometry']
    print(type(land_use_geoseries))
    # crs for PA South
    area_crs = 32129
    matched_tiles_polys = list()    
    lidar_tiles_polys.to_crs(area_crs)
    land_use_polys.to_crs(area_crs)

    for index, row in lidar_tiles_polys.iterrows():
       #print("row",row)

       pred_poly = row[3]
       #print("-----")
       #print("pred_poly",type(pred_poly), pred_poly)
       #print("shape",land_use_geoseries.shape)
       #print("++++++++++++")

       #print(land_use_geoseries.overlaps(pred_poly).any())
       if (land_use_geoseries.overlaps(pred_poly).any()==True) or (land_use_geoseries.contains(pred_poly).any()==True) or (land_use_geoseries.within(pred_poly).any()==True):

           print("match", row[0])  
           lidar_tiles_forested_polys_df = lidar_tiles_forested_polys_df.append(row, ignore_index=True)
       else:
           print("    no match", row[0])  
          

if not lidar_tiles_forested_polys_df.empty:
    lidar_tiles_forested_polys_df.to_file(lidar_tiles_forested_polys_fp)
    print("Total forested tiles:", len(lidar_tiles_forested_polys_df))
else:
    print("empty")
"""

# Wget the .laz files

+ Open the shape file containing the polygons of forested lidar tiles in Maryland.

+ For each row, wget the file at the url.

In [8]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=82b55b4a4b122ad323ba031cfa3c91b81f0e8206266a5b85f780531d88e9d86d
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [9]:
import os
import geopandas as gpd
import time

# import the wget module
from wget import download
#
# create a downloader class.
class downloader:
        
    # Create a downloadfile method
    # Accepting the url and the file storage location
    # Set the location to an empty string by default.

    def downloadFile(self, url, location=""):
         # Download file and with a custom progress bar
        download(url, out = location)

downloadObj = downloader()

lidar_tiles_forested_polys_fp = "/content/drive/MyDrive/crane_pennsylvania/pa_michaux_forest_area_rch/study_area_shapefile/pa_forest_district/pa_forest_district_01_tiles_32129.shp"

if os.path.exists(lidar_tiles_forested_polys_fp):
    lidar_tiles_forested_polys = gpd.read_file(lidar_tiles_forested_polys_fp)
    
    print("crs", lidar_tiles_forested_polys.crs)
    print(type(lidar_tiles_forested_polys))

    print("lidar_tiles_forested_polys",lidar_tiles_forested_polys.shape)
    # crs for Maryland
    area_crs = 26985
    lidar_tiles_forested_polys.to_crs(area_crs)

    for index, row in lidar_tiles_forested_polys.iterrows():
       #print("row",row)

       print(row[0],row[1],row[2])
       
       downloadObj.downloadFile(str(row[2]),"/content/drive/MyDrive/crane_pennsylvania/laz/pa_forest_district_01/")
       
       time.sleep(10)


crs epsg:32129
<class 'geopandas.geodataframe.GeoDataFrame'>
lidar_tiles_forested_polys (598, 7)
3624 2000_34002210pas11.laz https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/2574/2000_34002210pas11.laz

3629 2000_34002200pas11.laz https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/2574/2000_34002200pas11.laz

3636 2000_34002190pas11.laz https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/2574/2000_34002190pas11.laz

3645 2000_34002180pas04.laz https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/2574/2000_34002180pas04.laz

3651 2000_34002170pas04.laz https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/2574/2000_34002170pas04.laz

3659 2000_34002160pas04.laz https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/2574/2000_34002160pas04.laz

3806 2000_33002220pas11.laz https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/2574/2000_33002220pas11.laz

3815 2000_33002210pas11.laz https://noaa-nos-coastal-lid

In [ ]:
!wget --version

GNU Wget 1.19.4 built on linux-gnu.

-cares +digest -gpgme +https +ipv6 +iri +large-file -metalink +nls 
+ntlm +opie +psl +ssl/openssl 

Wgetrc: 
    /etc/wgetrc (system)
Locale: 
    /usr/share/locale 
Compile: 
    gcc -DHAVE_CONFIG_H -DSYSTEM_WGETRC="/etc/wgetrc" 
    -DLOCALEDIR="/usr/share/locale" -I. -I../../src -I../lib 
    -I../../lib -Wdate-time -D_FORTIFY_SOURCE=2 -DHAVE_LIBSSL -DNDEBUG 
    -g -O2 -fdebug-prefix-map=/build/wget-Xb5Z7Y/wget-1.19.4=. 
    -fstack-protector-strong -Wformat -Werror=format-security 
    -DNO_SSLv2 -D_FILE_OFFSET_BITS=64 -g -Wall 
Link: 
    gcc -DHAVE_LIBSSL -DNDEBUG -g -O2 
    -fdebug-prefix-map=/build/wget-Xb5Z7Y/wget-1.19.4=. 
    -fstack-protector-strong -Wformat -Werror=format-security 
    -DNO_SSLv2 -D_FILE_OFFSET_BITS=64 -g -Wall -Wl,-Bsymbolic-functions 
    -Wl,-z,relro -Wl,-z,now -lpcre -luuid -lidn2 -lssl -lcrypto -lpsl 
    ftp-opie.o openssl.o http-ntlm.o ../lib/libgnu.a 

Copyright (C) 2015 Free Software Foundation, Inc.
License 

In [ ]:
print(lidar_tiles_forested_polys_df)